In [1]:
from fancyimpute import MICE
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, SimpleFill
import pandas as pd
#import impyute as imp

Using TensorFlow backend.


In [2]:
data = pd.read_excel('OIC_SESRIC_28Aug2018_gender_dis.xlsx')

#Each year is a column. Converts it into a single column with all the years as rows
data=pd.melt(data, id_vars=['Country','Project Category','Indicator','Selected'], 
             value_vars=[2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017], var_name='Year')

#The Colum "Indicator" is pivoted into a column for each indicator
data=pd.pivot_table(data, values='value', index=['Country','Year'], columns=['Selected','Project Category','Indicator'])

#Sort values
data=data.sort_values(by=['Country','Year'])

#selects only the indicator with the appropriate flag (Selected=1)
data=data.xs(1, level='Selected', axis=1)
data

Project Category                  Economic Context                            \
Indicator                 Agriculture, Value Added Construction, Value Added   
Country              Year                                                      
Afghanistan          2008             2.987752e+09              8.704645e+08   
                     2009             3.911459e+09              9.926377e+08   
                     2010             4.462635e+09              1.238954e+09   
                     2011             5.171297e+09              1.394729e+09   
                     2012             5.416548e+09              1.595694e+09   
                     2013             5.331940e+09              1.695054e+09   
                     2014             5.136259e+09              1.877281e+09   
                     2015             4.482955e+09              2.032602e+09   
                     2016             4.663743e+09              1.868076e+09   
                     2017                      NaN                       NaN   
Albania              2008             2.168845e+09              2.037763e+09   
                     2009             2.022753e+09              1.801621e+09   
                     2010             2.141586e+09              1.541795e+09   
                     2011             2.349589e+09              1.639114e+09   
                     2012             2.312028e+09              1.403089e+09   
                     2013             2.499702e+09              1.314275e+09   
                     2014             2.644345e+09              1.139258e+09   
                     2015             2.274186e+09              1.017998e+09   
                     2016             2.356334e+09              1.011491e+09   
                     2017                      NaN                       NaN   
Algeria              2008             1.126326e+10              1.347090e+10   
                     2009             1.282013e+10              1.376587e+10   
                     2010             1.364852e+10              1.605294e+10   
                     2011             1.622224e+10              1.731016e+10   
                     2012             1.833592e+10              1.820006e+10   
                     2013             2.066321e+10              1.977252e+10   
                     2014             2.199332e+10              2.147209e+10   
                     2015             1.921812e+10              1.847029e+10   
                     2016             1.955629e+10              1.818297e+10   
                     2017                      NaN                       NaN   
...                                            ...                       ...   
United Arab Emirates 2008             2.358167e+09              3.181322e+10   
                     2009             2.375327e+09              3.139872e+10   
                     2010             2.212295e+09              3.184186e+10   
                     2011             2.356203e+09              3.153332e+10   
                     2012             2.393213e+09              3.069522e+10   
                     2013             2.511387e+09              3.148985e+10   
                     2014             2.578144e+09              3.329279e+10   
                     2015             2.653873e+09              3.476991e+10   
                     2016             2.795089e+09              3.565215e+10   
                     2017                      NaN                       NaN   
Uzbekistan           2008             5.838681e+09              1.657628e+09   
                     2009             6.306769e+09              2.286684e+09   
                     2010             7.112190e+09              2.555409e+09   
                     2011             8.214056e+09              2.811771e+09   
                     2012             9.199948e+09              3.283468e+09   
                     2013             1.011687e+10              3.754533e+09   


In [ ]:
from sklearn import preprocessing as pp
std_tmp=data
std_tmp=pp.StandardScaler().fit_transform(std_tmp)
std_tmp=pd.DataFrame(std_tmp,index=data.index, columns=data.columns)
std_tmp.describe()

In [3]:
std_tmp=data
std_nom_tmp=(std_tmp - std_tmp.min()) / (std_tmp.max() - std_tmp.min())
std_nom_tmp.describe()
std_tmp=std_nom_tmp

In [ ]:
std_tmp=data
std_tmp=pp.RobustScaler(quantile_range=(25.0, 75.0)).fit_transform(std_tmp)
std_tmp=pd.DataFrame(std_tmp,index=data.index, columns=data.columns)
std_nom_tmp=(std_tmp - std_tmp.min()) / (std_tmp.max() - std_tmp.min())
std_nom_tmp.describe()

In [ ]:
#standarises data

#try using standard scaler from sk-learn v0.20!!!!!!!
std_tmp=data
std_tmp=(std_tmp-std_tmp.min(axis=0))/(std_tmp.max(axis=0)-std_tmp.min(axis=0))
std_tmp_missing=std_tmp
std_tmp.isna().sum().sum()
std_tmp.describe()

In [50]:
#removing data to simulate
#todo!!!! test removing a full column
import random
import numpy as np
import random
print("NAN in original")
print(std_tmp.isna().sum().sum())  

std_tmp_missing=std_tmp.copy()
ix = [(row, col) for row in range(std_tmp_missing.shape[0]) for col in range(std_tmp_missing.shape[1])]
for row, col in random.sample(ix, int(round(.25*len(ix)))):
    std_tmp_missing.iat[row, col] = np.nan

print("NAN after mice")
print(std_tmp_missing.isna().sum().sum())  
    
    
#add full inducators in nan for some countries (random)
tmp=pd.DataFrame()
for name,group in std_tmp_missing.groupby("Country"):
    tmp9=group
    t=random.randint(0,100)
    if (t<80):
        t2=random.randint(0,len(group.columns.values)-1)
        while(group.iloc[:,t2].isna().sum()==10):
            print('already')
            t2=random.randint(0,len(group.columns.values)-1)
        else:
            group.iloc[:,t2]=np.nan
            #print(group.iloc[:,t2])
        tmp9=group
    
    tmp=tmp.append(tmp9, sort=True)
    
std_tmp_missing=tmp 

print("NAN after knn")
print(std_tmp_missing.isna().sum().sum())  

print("NAN in original")
print(std_tmp.isna().sum().sum())  

NAN in original
46712
NAN after mice
53887
already


/Users/manuelricardogalindo/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


already
already
already
already
already
already
already
already
already
already
already
already
already
already
already
already
already
already
already
already
already
already
already
already
already
NAN after knn
54076
NAN in original
46712


In [51]:
#mice and after knn
tmp=pd.DataFrame()
tmp_mice_knn=std_tmp_missing.copy()
for name, group in tmp_mice_knn.groupby('Country'):
    print(name)
    tmp2=group.dropna(axis=1,how='all')
    tmp3=MICE(visit_sequence='roman', init_fill_method='mean', n_imputations=10, n_burn_in=2, impute_type='col', verbose=False, min_value=0, max_value=1).complete(tmp2)
    tmp3=pd.DataFrame(tmp3,columns=tmp2.columns, index=tmp2.index)
    tmp=tmp.append(tmp3, sort=True)

#check how to avoid using itself as neighbour!!!!!!!!!!!!!!!!    
mice_knn=KNN(k=5, min_value=0, max_value=1).complete(tmp)
mice_knn=pd.DataFrame(mice_knn, columns=tmp.columns, index=tmp.index)


Afghanistan
Albania
Algeria
Azerbaijan
Bahrain
Bangladesh
Benin
Brunei
Burkina Faso
Cameroon
Chad
Comoros
Côte d'Ivoire
Djibouti
Egypt
Gabon
Gambia
Guinea
Guinea-Bissau
Guyana
Indonesia
Iran
Iraq
Jordan
Kazakhstan
Kuwait
Kyrgyzstan
Lebanon
Libya
Malaysia
Maldives
Mali
Mauritania
Morocco
Mozambique
Niger
Nigeria
Oman
Pakistan
Palestine
Qatar
Saudi Arabia
Senegal
Sierra Leone
Somalia
Sudan
Suriname
Syria
Tajikistan
Togo
Tunisia
Turkey
Turkmenistan
Uganda
United Arab Emirates
Uzbekistan
Yemen
Imputing row 1/570 with 81 missing, elapsed time: 0.389
Imputing row 101/570 with 69 missing, elapsed time: 0.540
Imputing row 201/570 with 36 missing, elapsed time: 0.757
Imputing row 301/570 with 82 missing, elapsed time: 0.883
Imputing row 401/570 with 22 missing, elapsed time: 1.042
Imputing row 501/570 with 30 missing, elapsed time: 1.223


In [52]:
#error mice_knn
(((std_tmp-mice_knn)**2).mean(0)**0.5).describe()

count    132.000000
mean       0.047053
std        0.054357
min        0.000000
25%        0.014637
50%        0.028987
75%        0.053367
max        0.354186
dtype: float64

In [53]:
#only median
tmp_median=std_tmp_missing.copy()
median=SimpleFill(fill_method='median').complete(tmp_median)
median=pd.DataFrame(median, columns=tmp_median.columns, index=tmp_median.index)


In [54]:
#error median
(((std_tmp-median)**2).mean(0)**0.5).describe()

count    132.000000
mean       0.110625
std        0.063942
min        0.000000
25%        0.072303
50%        0.105725
75%        0.137492
max        0.374634
dtype: float64

In [55]:
#linear+propagation+median
tpm_l_p_m=std_tmp_missing.copy()
l_p_m=tpm_l_p_m.groupby('Country').apply(
    lambda group:group.interpolate(method='linear', limit_area='inside'))
l_p_m=l_p_m.groupby('Country').apply(
    lambda group:group.fillna(method='ffill')) 
l_p_m=l_p_m.groupby('Country').apply(
    lambda group:group.fillna(method='bfill')) 

l_p_m=l_p_m.fillna(l_p_m.median())

In [56]:
#error linear+propagation+median
(((std_tmp-l_p_m)**2).mean(0)**0.5).describe()

count    132.000000
mean       0.052367
std        0.065826
min        0.000000
25%        0.011655
50%        0.027804
75%        0.071089
max        0.374634
dtype: float64

In [57]:
#linear+propagation+knn
tpm_l_p_knn=std_tmp_missing.copy()
l_p_knn=tpm_l_p_knn.groupby('Country').apply(
    lambda group:group.interpolate(method='linear', limit_area='inside'))
l_p_knn=l_p_knn.groupby('Country').apply(
    lambda group:group.fillna(method='ffill')) 
l_p_knn=l_p_knn.groupby('Country').apply(
    lambda group:group.fillna(method='bfill')) 

#try replacing mean by knn -->>>#check how to avoid using itself as neighbour!!!!!!!!!!!!!!!! 
l_p_knn=KNN(k=5, min_value=0, max_value=1).complete(l_p_knn)
l_p_knn=pd.DataFrame(l_p_knn, columns=tmp.columns, index=tmp.index)

Imputing row 1/570 with 81 missing, elapsed time: 0.378
Imputing row 101/570 with 69 missing, elapsed time: 0.541
Imputing row 201/570 with 36 missing, elapsed time: 0.769
Imputing row 301/570 with 82 missing, elapsed time: 0.900
Imputing row 401/570 with 22 missing, elapsed time: 1.070
Imputing row 501/570 with 30 missing, elapsed time: 1.262


In [58]:
#error linear+propagation+knn
(((std_tmp-l_p_knn)**2).mean(0)**0.5).describe()

count    132.000000
mean       0.045774
std        0.055426
min        0.000000
25%        0.011405
50%        0.026344
75%        0.061071
max        0.341806
dtype: float64

In [59]:
#linear+propagation+mean
tpm_l_p_mean=std_tmp_missing.copy()
l_p_mean=tpm_l_p_mean.groupby('Country').apply(
    lambda group:group.interpolate(method='linear', limit_area='inside'))
l_p_mean=l_p_mean.groupby('Country').apply(
    lambda group:group.fillna(method='ffill')) 
l_p_mean=l_p_mean.groupby('Country').apply(
    lambda group:group.fillna(method='bfill')) 

#try replacing mean by knn -->>>#check how to avoid using itself as neighbour!!!!!!!!!!!!!!!! 
l_p_mean=l_p_mean.fillna(l_p_m.mean())

In [60]:
#error linear+propagation+mean
(((std_tmp-l_p_mean)**2).mean(0)**0.5).describe()

count    132.000000
mean       0.050205
std        0.059529
min        0.000000
25%        0.012363
50%        0.028270
75%        0.069194
max        0.330174
dtype: float64

In [61]:
#try knn and mice
#try other algorithms from impyute